In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
Idle_data = pd.read_csv('..\HandTrackingData\IdleData.csv')
Idle_data = Idle_data.to_numpy()

Pistol_data = pd.read_csv('..\HandTrackingData\PistalData.csv')
Pistol_data = Pistol_data.to_numpy()

Drill_data = pd.read_csv('..\HandTrackingData\DrillData.csv')
Drill_data = Drill_data.to_numpy()

Sword_data = pd.read_csv('..\HandTrackingData\SwordData.csv')
Sword_data = Sword_data.to_numpy()

Dagger_data = pd.read_csv('..\HandTrackingData\DaggerData.csv')
Dagger_data = Dagger_data.to_numpy()

KitchenKnife_data = pd.read_csv('..\HandTrackingData\KitchenKnifeData.csv')
KitchenKnife_data = KitchenKnife_data.to_numpy()

print(Idle_data.shape)
print(Pistol_data.shape)
print(Drill_data.shape)
print(Sword_data.shape)
print(Dagger_data.shape)
print(KitchenKnife_data.shape)

(2000, 139)
(2000, 139)
(2000, 139)
(2000, 139)
(2000, 139)
(2000, 139)


In [3]:
Idle_data_Seq= []
Pistol_data_Seq = []
Drill_data_Seq = []
Sword_data_Seq = []
Dagger_data_Seq = []
KitchenKnife_data_Seq = []


Objects = ['Idle','Pistol', 'Drill', 'Sword', 'Dagger', 'KitchenKnife']
WindowSize = 80
#secs_for_action = 100

In [4]:

def Create_Sequences(data, window_size):
    """
    주어진 데이터와 윈도우 크기를 기반으로 시퀀스를 생성합니다.

    Args:
        data (list or np.ndarray): 입력 데이터
        window_size (int): 시퀀스의 윈도우 크기

    Returns:
        np.ndarray: 생성된 시퀀스 배열
    """
    
    Sequences = []
    for seq in range(len(data) - window_size):
        Sequences.append(data[seq:seq + window_size])
    return np.array(Sequences)

In [5]:
Idle_data_Seq = Create_Sequences(Idle_data, WindowSize)
Pistol_data_Seq = Create_Sequences(Pistol_data, WindowSize)
Drill_data_Seq = Create_Sequences(Drill_data, WindowSize)
Sword_data_Seq = Create_Sequences(Sword_data, WindowSize)
Dagger_data_Seq = Create_Sequences(Dagger_data, WindowSize)
KitchenKnife_data_Seq = Create_Sequences(KitchenKnife_data, WindowSize)

In [6]:
print(Idle_data_Seq.shape)
print(Pistol_data_Seq.shape)
print(Drill_data_Seq.shape)
print(Sword_data_Seq.shape)
print(Dagger_data_Seq.shape)
print(KitchenKnife_data_Seq.shape)

(1920, 80, 139)
(1920, 80, 139)
(1920, 80, 139)
(1920, 80, 139)
(1920, 80, 139)
(1920, 80, 139)


In [7]:
ConcatenatedSeqData = np.concatenate([Idle_data_Seq, Pistol_data_Seq , Drill_data_Seq, Sword_data_Seq, Dagger_data_Seq, KitchenKnife_data_Seq], axis=0)

ConcatenatedSeqData.shape

(11520, 80, 139)

In [8]:
# 이거 임시로 -2로 둔거, 마지막 데이터 고치면 -1로 바꿔야함.
X_data = ConcatenatedSeqData[:, :, :-1]
Labels = ConcatenatedSeqData[:, 0, -1]

#Labels = np.zeros_like(Labels)

#print(X_data.shape)
#print(X_data)

#print(Labels.shape)
print(Labels)

[0. 0. 0. ... 5. 5. 5.]


In [9]:
from tensorflow.keras.utils import to_categorical

Y_data = to_categorical(Labels, num_classes=len(Objects))

print(Y_data)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


In [10]:
from sklearn.model_selection import train_test_split

X_data = X_data.astype(np.float32)
Y_data = Y_data.astype(np.float32)

X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=2024)

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

(9216, 80, 138) (9216, 6)
(2304, 80, 138) (2304, 6)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    #LSTM(64, activation='tanh', input_shape=X_train.shape[1:3]),
    LSTM(64, activation='tanh', input_shape=(80, 138)),
    Dense(32, activation='tanh'),
    Dense(len(Objects), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                51968     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 54,246
Trainable params: 54,246
Non-trainable params: 0
_________________________________________________________________


In [12]:
print("입력 형식:", model.input_shape)

입력 형식: (None, 80, 138)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    X_train,
    Y_train,
    validation_data=(X_val, Y_val),
    epochs=100,
    callbacks=[
        ModelCheckpoint('models/model3.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
import tensorflow as tf

# .h5 모델 로드
h5_model_path = "models\model3.h5"
saved_model_path = "models\model3_tf"

model = tf.keras.models.load_model(h5_model_path)

# SavedModel 형식으로 저장
tf.saved_model.save(model, saved_model_path)

print(f"SavedModel saved at: {saved_model_path}")

#tf python -m tf2onnx.convert --saved-model C:\Project\saved_model --output C:\Project\model2.onnx
#python -m tf2onnx.convert --saved-model tensorflow-model-path --output model.onnx
#python -m tf2onnx.convert --input model3.h5 --output model33.onnx
#python -m tf2onnx.convert --saved-model path/to/saved_model --output model.onnx --input-shape "1,80,138"

# 사용해서 onnx 파일 생성

INFO:tensorflow:Assets written to: models\model3_tf\assets


INFO:tensorflow:Assets written to: models\model3_tf\assets


SavedModel saved at: models\model3_tf


In [ ]:
import onnx

# onnx 파일 구조 파악.
onnx_model = onnx.load("D:\GripGenObject\LSTMModel\models\model3.onnx")
onnx.checker.check_model(onnx_model)

print(onnx.helper.printable_graph(onnx_model.graph))

In [ ]:
import onnx

# onnx 파일 구조 파악.
onnx_model = onnx.load("C:\Project\[KHU-GameEngineering]_LSTM_Hand_Pred\KHU-GameEngineering-HandsPred\model12.onnx")
onnx.checker.check_model(onnx_model)

print(onnx.helper.printable_graph(onnx_model.graph))